In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import argparse
from collections import deque
from utility_v2 import *
from trfl import indexing_ops
import trfl

2024-01-07 08:29:44.453771: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-07 08:29:44.453823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-07 08:29:44.455774: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-07 08:29:44.466681: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-07 08:29:45.672441: W tensorflow/compiler/tf2

In [2]:
epoch = 30
data = "abc"
batch_size = 256
hidden_factor = 62
r_click = 1
r_buy = 5
lr = 0.01
discount = 0.5

In [3]:
class QNetwork(tf.keras.Model):
    def __init__(self, hidden_size, learning_rate, item_num, state_size, pretrain, model_name='DQNetwork'):
        super(QNetwork, self).__init__(name=model_name)
        # self.state_size = state_size
        # learning_rate = learning_rate
        self.hidden_size = hidden_size
        self.item_num = int(item_num)
        self.pretrain = pretrain
        self.model_name = model_name

        self.all_embeddings = tf.keras.layers.Embedding(input_dim=self.item_num + 1, output_dim=self.hidden_size, name='all_embeddings')
        self.gru = tf.keras.layers.GRU(self.hidden_size, name='gru')

        self.output1 = tf.keras.layers.Dense(self.item_num, activation=None, name="q-value")
        self.output2 = tf.keras.layers.Dense(self.item_num, activation=None, name="ce-logits")

    def call(self, inputs, len_state=10):
        inputs_tensor = tf.convert_to_tensor(inputs)
        input_emb = self.all_embeddings(inputs_tensor)
        states_hidden = self.gru(input_emb)

        q_values = self.output1(states_hidden)
        ce_logits = self.output2(states_hidden)
        
        return q_values, ce_logits



In [4]:
def evaluate(mainQN):
    
    print("Entered evaluate part")
    
    eval_sessions = pd.read_pickle(os.path.join(data_directory, 'sampled_val.df'))
    eval_ids = eval_sessions.session_id.unique()
    groups = eval_sessions.groupby('session_id')
    batch = 100
    evaluated = 0
    total_clicks = 0.0
    total_purchase = 0.0
    total_reward = [0, 0, 0, 0]
    hit_clicks = [0, 0, 0, 0]
    ndcg_clicks = [0, 0, 0, 0]
    hit_purchase = [0, 0, 0, 0]
    ndcg_purchase = [0, 0, 0, 0]
    
    while evaluated < len(eval_ids):
        states, len_states, actions, rewards = [], [], [], []
        for i in range(batch):
            id = eval_ids[evaluated]
            group = groups.get_group(id)
            history = []
            for index, row in group.iterrows():
                state = list(history)
                len_states.append(state_size if len(state) >= state_size else 1 if len(state) == 0 else len(state))
                state = pad_history(state, state_size, item_num)
                states.append(state)
                action = row['item_id']
                is_buy = row['is_buy']
                reward = reward_buy if is_buy == 1 else reward_click
                if is_buy == 1:
                    total_purchase += 1.0
                else:
                    total_clicks += 1.0
                actions.append(action)
                rewards.append(reward)
                history.append(row['item_id'])
            evaluated += 1

        _, prediction = mainQN(states)
        sorted_list = np.argsort(prediction)
        calculate_hit(sorted_list, topk, actions, rewards, reward_click, total_reward, hit_clicks, ndcg_clicks, hit_purchase, ndcg_purchase)
    
    print('#############################################################')
    print('total clicks: %d, total purchase:%d' % (total_clicks, total_purchase))
    for i in range(len(topk)):
        hr_click = hit_clicks[i] / total_clicks
        hr_purchase = hit_purchase[i] / total_purchase
        ng_click = ndcg_clicks[i] / total_clicks
        ng_purchase = ndcg_purchase[i] / total_purchase
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        print('cumulative reward @ %d: %f' % (topk[i], total_reward[i]))
        print('clicks hr ndcg @ %d : %f, %f' % (topk[i], hr_click, ng_click))
        print('purchase hr and ndcg @%d : %f, %f' % (topk[i], hr_purchase, ng_purchase))
    print('#############################################################')

In [5]:
# class QNetwork(tf.keras.Model):
#     def __init__(self, hidden_size, learning_rate, item_num, state_size, pretrain, model_name='DQNetwork'):
#         super(QNetwork, self).__init__(name=model_name)
#         # self.state_size = state_size
#         # self.learning_rate = learning_rate
#         self.hidden_size = hidden_size
#         self.item_num = int(item_num)
#         self.pretrain = pretrain
#         self.model_name = model_name

#         self.all_embeddings = tf.keras.layers.Embedding(input_dim=self.item_num + 1, output_dim=self.hidden_size, name='all_embeddings')
#         self.gru = tf.keras.layers.GRU(self.hidden_size, name='gru')


#         # self.inputs = tf.keras.layers.Input(shape=(state_size,), dtype=tf.int32)
#         # self.len_state = tf.keras.layers.Input(shape=(), dtype=tf.int32)
#         # self.input_emb = tf.keras.layers.Embedding(input_dim=self.item_num + 1, output_dim=self.hidden_size)(self.inputs)
#         # gru_out, self.states_hidden = tf.keras.layers.GRU(self.hidden_size)(self.input_emb, mask=tf.sequence_mask(self.len_state, dtype=tf.float32))

#         self.output1 = tf.keras.layers.Dense(self.item_num, activation=None, name="q-value")(self.states_hidden)
#         self.output2 = tf.keras.layers.Dense(self.item_num, activation=None, name="ce-logits")(self.states_hidden)

#         # self.actions = tf.keras.layers.Input(shape=(), dtype=tf.int32)
#         # self.targetQs_ = tf.keras.layers.Input(shape=(item_num,), dtype=tf.float32)
#         # self.targetQs_selector = tf.keras.layers.Input(shape=(item_num,), dtype=tf.float32)
#         # self.reward = tf.keras.layers.Input(shape=(), dtype=tf.float32)
#         # self.discount = tf.keras.layers.Input(shape=(), dtype=tf.float32)

#         def forward(self, inputs, len_state):
#             inputs = tf.convert_to_tensor(inputs)
#             input_emb = self.all_embeddings(inputs)
#             states_hidden = self.gru(input_emb)

#             q_values = self.output1(states_hidden)
#             ce_logits = self.output2(states_hidden)

#             # qloss, q_learning = trfl.double_qlearning(Q_values, self.actions, self.reward, self.discount, self.targetQs_, self.targetQs_selector)
#             # q_indexed = tf.gather(Q_values, self.actions, axis=1, batch_dims=0)
#             # celoss = tf.multiply(q_indexed, tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.actions, logits=ce_logits))
#             # loss = tf.reduce_mean(qloss + celoss)
            
#             return ce_logits, q_values


In [6]:
data_directory = r'E:\MIni\v2\data'
data_directory = '/media/amd/New Volume/MIni/v2/data'
data_statis = pd.read_pickle(os.path.join(data_directory, 'data_statis.df'))
state_size = data_statis['state_size'][0]
item_num = data_statis['item_num'][0]
reward_click = r_click
reward_buy = r_buy
topk = [5, 10, 15, 20]

In [7]:
QN_1 = QNetwork(hidden_size=hidden_factor, learning_rate=lr, item_num=item_num,
                    state_size=state_size, pretrain=False)
QN_2 = QNetwork(hidden_size=hidden_factor, learning_rate=lr, item_num=item_num,
                    state_size=state_size, pretrain=False)

2024-01-07 08:29:48.628117: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [10]:
replay_buffer = pd.read_pickle(os.path.join(data_directory, 'replay_buffer.df'))
total_step = 0

optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

num_rows=replay_buffer.shape[0]
num_batches=int(num_rows/batch_size)

#current_q_val = tf.Variable(tf.zeros([batch_size, item_num]))

# optimizer.build(mainQN.trainable_variables)
optimizer.build(QN_1.trainable_variables + QN_2.trainable_variables)  # Build optimizer with all trainable variables

for i in range(epoch):
    for j in range(num_batches):
        batch = replay_buffer.sample(n=batch_size).to_dict()
        
        # print(batch.keys())
        
        state = list(batch['state'].values())
        len_state = list(batch['len_state'].values())
        
        next_state = list(batch['next_state'].values())
        len_next_state = list(batch['len_next_states'].values())
        
        action = list(batch['action'].values())
        is_buy = list(batch['is_buy'].values())
        reward = [reward_buy if is_buy[index] else reward_click for index in range(len(is_buy))]
        
        #This is giving raw value of discount need to change it to add a variable
        discounts = [discount] * len(action)

        pointer = np.random.randint(0, 2)
        if pointer == 0:
            mainQN = QN_1
            target_QN = QN_2
        else:
            mainQN = QN_2
            target_QN = QN_1
        
        action = tf.convert_to_tensor(action)  # Convert to TensorFlow tensor
        reward = tf.convert_to_tensor(reward, dtype=tf.float32)  # Convert to TensorFlow tensor
        # discount = tf.ones_like(reward) * discount
        discounts = tf.convert_to_tensor(discounts)  # Convert to TensorFlow tensor
        
        with tf.GradientTape() as tape:
            
            current_Qs, ce_logits = mainQN(state)
            
            target_Qs, _ = target_QN(next_state)
            target_Qs_selector, _ = mainQN(next_state)
            # print(target_Qs.shape, target_Qs_selector.shape, current_Qs.shape, ce_logits.shape, "the shapes")

            is_done = list(batch['is_done'].values())
            for index in range(target_Qs.shape[0]):
                if is_done[index]:
                    # target_Qs[index] = np.zeros(item_num)
                    target_Qs = tf.tensor_scatter_nd_update(target_Qs, tf.expand_dims(tf.range(tf.shape(target_Qs)[0]), axis=1), tf.zeros([tf.shape(target_Qs)[0], item_num]))


            # print("current_Qs:", current_Qs.dtype, current_Qs.shape)
            # print("action:", action.dtype, action.shape)
            # print("reward:", reward.dtype, reward.shape)
            # print("discount:", discount.dtype, discount.shape)
            # print("target_Qs:", target_Qs.dtype, target_Qs.shape)
            # print("target_Qs_selector:", target_Qs_selector.dtype, target_Qs_selector.shape)
            
            qloss, q_learning = trfl.double_qlearning(current_Qs, action, reward, discounts,
                                                      target_Qs, target_Qs_selector)
            
            q_indexed = tf.stop_gradient(indexing_ops.batched_index(current_Qs, action))
            celoss = tf.multiply(q_indexed, tf.nn.sparse_softmax_cross_entropy_with_logits(labels=action,
                                                                                           logits=ce_logits))
            loss = tf.reduce_mean(qloss + celoss)

        grads = tape.gradient(loss, mainQN.trainable_variables)
        optimizer.apply_gradients(zip(grads, mainQN.trainable_variables))

        total_step += 1
        if total_step % 100 == 0:
            print('Epoch {} - Step {} - Loss {}'.format(i, total_step, loss.numpy()))
        if total_step % 1000 == 0:
            QN_1.save_weights('DQN_1.h5')
            QN_2.save_weights('DQN_2.h5')
            evaluate(mainQN)


it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works
it works
aaaaaaait works


KeyboardInterrupt: 